In [1]:
#%matplotlib nbagg

import geofield
import dipole
import scipy, numpy
import newleg
import xyzfield
from matplotlib import pyplot, rcParams
import matplotlib
rcParams.update({'font.family': "Source Sans Pro"})
rcParams.update({'font.size': 14})

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [15]:
import bootstrap
import importlib; importlib.reload(bootstrap)

<module 'bootstrap' from '/home/yo/Programs/python/geofieldpy/bootstrap.py'>

In [16]:
sha=geofield.SHADIFData("../data/sha.dif.14k/coef_EPSL_tnmgh.dat", "../data/sha.dif.14k/error_coef_EPSL_tnmgh.dat")

In [31]:
(x,y,z), (x_dev, y_dev, z_dev)=bootstrap.bootstrap(dipole.ecc_dipole, (sha.gcomp[... , :8],), errors=(sha.gcomp_err[..., :8],))

In [23]:
pyplot.show(pyplot.plot(sha.years, z,
                        sha.years, z+z_dev,
                        sha.years, z-z_dev))

In [33]:
def szrphi(x,y,z):
    s, zz, phi = dipole.xyz2cyl(x,y,z)
    r = numpy.sqrt(x**2 + y**2 + z**2)
    return s, zz, r, phi

#wow this is cool as heck
bigfunc = lambda g: szrphi(*dipole.ecc_dipole(g))

In [42]:
(s,z,r, phi), (s_dev, z_dev, r_dev, phi_dev) = bootstrap.bootstrap(bigfunc, (sha.gcomp[..., :8],), errors=(sha.gcomp_err[..., :8],))

In [44]:
fig, ax = pyplot.subplots(figsize=(20,10))

ax.plot(sha.years, s, color="green", label="s")
ax.plot(sha.years, z, "m-", label="z")
ax.plot(sha.years, r, "b-", label="r")

ax.fill_between(sha.years, r-r_dev, r+r_dev, color="blue", lw=0, alpha=0.07)
ax.fill_between(sha.years, z-z_dev, z+z_dev, color="magenta", lw=0, alpha=0.07)
ax.fill_between(sha.years, s-s_dev, s+s_dev, color="green", lw=0, alpha=0.07)

ax.set_xlabel("Years AD")
ax.set_ylabel("s,z,r (km)")
ax.legend()

ax.set_xbound(-1000,sha.years[-1])

pyplot.show(fig)

In [50]:
#this is fuckingf expensive my dude, better run it on a BIG COMPUTE
(nlon_1, nlat_1), (nlon_1_dev, nlat_1_dev) = bootstrap.bootstrap(xyzfield.magnetic_north, args=(sha.gcomp,), errors=(sha.gcomp_err,), iterations=100, order=1, south=False)
#other things to bootstrap:
# - the rest of the poles (potentially forever)
# - average intensity (mildly expensive, just take less points)
# - pqpd (not expensive)
# - curvature (just re-bootstrap from the pole calculations)
# - paris curve

In [ ]:
fig, (ax1, ax2) = pyplot.subplots(2,1)

ax1.plot(sha.years, nlon_1, "b-")
ax1.plot(sha.years, nlon_1+nlon_1_dev, "b--")
ax1.plot(sha.years, nlon_1-nlon_1_dev, "b--")

ax2.plot(sha.years, nlat_1, "b-")
ax2.plot(sha.years, nlat_1+nlat_1_dev, "b--")
ax2.plot(sha.years, nlat_1-nlat_1_dev, "b--")
